# Ambiente de Configuração

## Definir variáveis do ambiente

In [ ]:
isAtGoogleColab = False

## Instalando bibliotecas

In [ ]:
%pip install pandas
%pip install seaborn
%pip install matplotlib.pyplot
%pip install numpy
%pip install nltk
%pip install install openpyxl
%pip install spacy
%pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz

## Importando bibliotecas

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import nltk
import spacy
from nltk.corpus import stopwords

## Baixando dados do spacy

In [ ]:
spacy.cli.download('pt_core_news_lg')

## Importando dados de treino

In [ ]:
data = pd.read_csv('../data/classification-labeled.csv', encoding='latin1', delimiter=';', on_bad_lines='skip')

# Análise Descritiva

## Separar as palavras dos dados

Contando os caracteres

In [ ]:
df["qt_char"] = df['comment'].apply(len)
df.head()

Contando as palavras

## Estatísticas gerais

## Gráficos

In [ ]:
nlp = spacy.load('en_core_web_sm')

def remove_stopwords(text):
    """
    Removes stopwords, punctuation, and spaces from the input text using spaCy.
    """
    doc = nlp(text)
    filtered_sentence = [token.text for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
    return " ".join(filtered_sentence)

data['processed_comment'] = data['comment'].apply(remove_stopwords)

"""
Finds the ten most common words in the processed comments.
"""
all_words = " ".join(data['processed_comment']).split()
common_words_counts = Counter(all_words).most_common(10)

"""
Prepares data for a bubble chart by calculating the average word count, median sentiment,
and frequency for each of the ten most common words.
"""
word_details = []
for word, count in common_words_counts:
    temp_data = data[data['comment'].str.contains(word, case=False, regex=False)]
    avg_word_count = temp_data['comment'].apply(lambda x: len(x.split())).mean()
    median_sentiment = temp_data['sentiment'].median()
    frequency = temp_data.shape[0]
    word_details.append({'word': word, 'avg_word_count': avg_word_count, 'median_sentiment': median_sentiment, 'frequency': frequency})

word_df = pd.DataFrame(word_details)

plt.figure(figsize=(14, 10))
palette = sns.color_palette("hsv", n_colors=len(word_df['word']))
bubble = sns.scatterplot(data=word_df, x='avg_word_count', y='median_sentiment', size='frequency', sizes=(100, 2000), hue='word', alpha=0.7, palette=palette)

"""
Adjusts text labels and customizes the legends to avoid overlapping and provide clear visualization.
Includes separate legends for circle sizes explaining their representation.
"""
for i, row in word_df.iterrows():
    plt.annotate(f"{row['word']} ({row['frequency']})",
                 xy=(row['avg_word_count'], row['median_sentiment']),
                 xytext=(row['avg_word_count'] + np.random.uniform(0.5, 1.5), row['median_sentiment'] + np.random.uniform(-0.05, 0.05)),
                 textcoords="data",
                 ha='right', va='center',
                 arrowprops=dict(arrowstyle="->", connectionstyle="arc3", color='gray', lw=0.5))

handles, labels = bubble.get_legend_handles_labels()
plt.legend(handles=handles, labels=labels, title="Words", loc='upper right', bbox_to_anchor=(1.25, 1))

legend_elements = [Patch(facecolor='grey', edgecolor='grey',
                         label='Circle size represents frequency of word usage')]
plt.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.25, 0.85))

plt.title('Improved Bubble Chart of Word Frequency, Average Comment Length, and Median Sentiment')
plt.xlabel('Average Number of Words')
plt.ylabel('Median Sentiment')
plt.grid(True)
plt.show()



O gráfico "Improved Bubble Chart of Word Frequency, Average Comment Length, and Median Sentiment" utiliza a frequência de palavras, o comprimento médio dos comentários e o sentimento mediano para ilustrar como diferentes termos são utilizados nos comentários. Nele, termos como "Uber" destacam-se pela alta frequência e por aparecerem em comentários mais longos, sugerindo uma relevância ou interesse significativo. Palavras como "lobbied" e "leak" apresentam sentimentos predominantemente negativos, refletindo possíveis contextos críticos ou problemáticos. Este gráfico oferece uma visão clara de quais temas são mais discutidos ou controversos, ajudando na identificação de padrões importantes nos dados.

In [ ]:
"""
Preprocess the text by converting all comments to lowercase, removing all punctuation, and digits.
"""
data['comment'] = data['comment'].str.lower().str.replace('[^\w\s]', '', regex=True).str.replace('\d+', '', regex=True)

"""
Define a set of stop words to be excluded from the analysis. This set can be updated based on specific requirements.
"""
stop_words = set(['and', 'the', 'uber', 'to', 'of', 'in', 'i', 'a', 'for', 'is', 'it', 'that', 'how', 'you'])

def get_top_words(data, sentiment, top_n=10):
    """
    Retrieves the top 'n' words for a given sentiment, excluding predefined stop words.
    Args:
        data (DataFrame): DataFrame containing the comments and their associated sentiments.
        sentiment (mixed): The sentiment category for which to retrieve top words.
        top_n (int): Number of top words to retrieve.

    Returns:
        list: A list of the most common words for the specified sentiment, excluding stop words.
    """
    subset = data[data['sentiment'] == sentiment]
    words = " ".join(subset['comment']).split()
    filtered_words = [word for word in words if word not in stop_words]
    return [word for word, count in Counter(filtered_words).most_common(top_n)]

"""
Prepare and collect the top words for each unique sentiment in the dataset for further analysis.
"""
sentiments = data['sentiment'].unique()
top_words_by_sentiment = {sent: get_top_words(data, sent) for sent in sentiments}

"""
Calculate and compile data concerning the frequency of each top word across all sentiments.
"""
rows = []
for sentiment, top_words in top_words_by_sentiment.items():
    for word in top_words:
        for sent in sentiments:
            subset = data[data['sentiment'] == sent]
            count = sum(subset['comment'].str.split().apply(lambda x: x.count(word)))
            rows.append({'Word': word, 'Sentiment': sent, 'Frequency': count})

"""
Convert the compiled data into a DataFrame for visualization purposes.
"""
plot_data = pd.DataFrame(rows)

def create_plot(plot_data, focus_sentiment):
    """
    Generates a bar plot for word frequencies focused on a specific sentiment.
    Args:
        plot_data (DataFrame): DataFrame containing the words, their frequencies, and sentiments.
        focus_sentiment (mixed): The sentiment to focus on for the plot.

    Displays:
        A bar plot visualizing the frequency of words focused on the specified sentiment.
    """
    focused_words = top_words_by_sentiment[focus_sentiment]
    focused_plot_data = plot_data[plot_data['Word'].isin(focused_words)]
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Word', y='Frequency', hue='Sentiment', data=focused_plot_data, palette='viridis')
    plt.title(f'Word Frequencies Focused on Sentiment {focus_sentiment} (Excluding Common Words)')
    plt.xticks(rotation=45)
    plt.show()

"""
Generate and display plots for each sentiment represented in the data.
"""
for sent in sentiments:
    create_plot(plot_data, sent)


Os gráficos mostram as frequências de palavras focadas em diferentes sentimentos após a remoção manual de stopwords comuns. Nos gráficos para sentimentos neutro e positivo, as palavras que emergem refletem aspectos mais específicos dos comentários, sem a interferência de termos gerais. Por exemplo, palavras como "driver" e "eats" são proeminentes em comentários positivos, apontando para tópicos específicos relacionados ao serviço. Já para o sentimento neutro, termos como "have" e "but" sugerem um contexto mais equilibrado ou discursivo. Essas análises ajudam a identificar quais aspectos dos serviços ou produtos estão recebendo atenção em diferentes tonalidades emocionais, permitindo uma compreensão mais precisa do feedback dos usuários.

In [ ]:
"""
Preprocess the text by converting all comments to lowercase, removing punctuation, and digits.
"""
data['comment'] = data['comment'].str.lower().str.replace('[^\w\s]', '', regex=True).str.replace('\d+', '', regex=True)

"""
Define a set of English stopwords using the NLTK library.
"""
stop_words = set(stopwords.words('english'))

def get_top_words(data, sentiment, top_n=10):
    """
    Retrieves the top 'n' words for a given sentiment, excluding stopwords.
    Args:
        data (DataFrame): DataFrame containing the comments and their associated sentiments.
        sentiment (mixed): The sentiment category to filter the comments.
        top_n (int): Number of top words to retrieve.

    Returns:
        list: A list of the most common words for the specified sentiment, excluding stopwords.
    """
    subset = data[data['sentiment'] == sentiment]
    words = " ".join(subset['comment']).split()
    filtered_words = [word for word in words if word not in stop_words]
    return [word for word, count in Counter(filtered_words).most_common(top_n)]

"""
Prepare and collect the top words for each unique sentiment in the dataset for further analysis.
"""
sentiments = data['sentiment'].unique()
top_words_by_sentiment = {sent: get_top_words(data, sent) for sent in sentiments}

"""
Calculate and compile data concerning the frequency of each top word across all sentiments.
"""
rows = []
for sentiment, top_words in top_words_by_sentiment.items():
    for word in top_words:
        for sent in sentiments:
            subset = data[data['sentiment'] == sent]
            count = sum(subset['comment'].str.split().apply(lambda x: x.count(word)))
            rows.append({'Word': word, 'Sentiment': sent, 'Frequency': count})

"""
Convert the compiled data into a DataFrame for visualization purposes.
"""
plot_data = pd.DataFrame(rows)

def create_plot(plot_data, focus_sentiment):
    """
    Generates a bar plot for word frequencies focused on a specific sentiment.
    Args:
        plot_data (DataFrame): DataFrame containing the words, their frequencies, and sentiments.
        focus_sentiment (mixed): The sentiment to focus on for the plot.

    Displays:
        A bar plot visualizing the frequency of words focused on the specified sentiment.
    """
    focused_words = top_words_by_sentiment[focus_sentiment]
    focused_plot_data = plot_data[plot_data['Word'].isin(focused_words)]
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Word', y='Frequency', hue='Sentiment', data=focused_plot_data, palette='viridis')
    plt.title(f'Word Frequencies Focused on Sentiment {focus_sentiment} (Excluding NLTK Stopwords)')
    plt.xticks(rotation=45)
    plt.show()

"""
Generate and display plots for each sentiment represented in the data.
"""
for sent in sentiments:
    create_plot(plot_data, sent)

Os gráficos visualizam a frequência de palavras agrupadas por sentimentos negativo, neutro e positivo, com foco em termos que excluem stopwords da NLTK. A palavra "Uber" é preeminente em todos os sentimentos, destacando-se especialmente nos comentários negativos e neutros, sugerindo sua relevância frequente nas discussões. Termos como "leak" e "secretly" nos negativos indicam tópicos de críticas ou controvérsias. Em sentimentos neutros, palavras como "driver" e "time" aparecem, refletindo discussões mais neutras ou informativas. Nos positivos, além de "Uber", "driver" também é frequente, apontando para comentários favoráveis ou positivos sobre o serviço. Este padrão de palavras ajuda a identificar os principais temas discutidos e o tom geral dos comentários em diferentes sentimentos.

In [ ]:
"""
Preprocess the text by converting all comments to lowercase, removing punctuation and digits,
and calculating the word count for each comment.
"""
data['comment'] = data['comment'].str.lower().str.replace('[^\w\s]', '', regex=True).str.replace('\d+', '', regex=True)
data['comment_length'] = data['comment'].apply(lambda x: len(x.split()))

"""
Calculate the length of each comment in characters.
"""
data['character_length'] = data['comment'].apply(len)

"""
Visualize the distribution of comment lengths in words across different sentiments using a boxplot.
"""
plt.figure(figsize=(10, 6))
sns.boxplot(x='sentiment', y='comment_length', data=data)
plt.title('Comment Length Distribution by Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Number of Words')
plt.show()

"""
Visualize the distribution of comment lengths in words across different sentiments using a scatter plot,
which is useful for observing the spread of data points.
"""
plt.figure(figsize=(10, 6))
sns.stripplot(x='sentiment', y='comment_length', data=data, jitter=True, alpha=0.3)
plt.title('Comment Length Scatter by Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Number of Words')
plt.show()

"""
Visualize the distribution of character lengths in comments across different sentiments using a bar plot.
"""
plt.figure(figsize=(10, 6))
sns.barplot(x='sentiment', y='character_length', data=data, ci=None)  # ci=None removes the error bars
plt.title('Character Length Distribution by Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Number of Characters')
plt.show()

"""
Visualize the distribution of character lengths in comments across different sentiments using a scatter plot,
providing a granular view of the distribution.
"""
plt.figure(figsize=(10, 6))
sns.stripplot(x='sentiment', y='character_length', data=data, jitter=True, alpha=0.3)
plt.title('Character Length Scatter by Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Number of Characters')
plt.show()

Análise de Comprimento dos Comentários por Sentimento (Palavras):
Os gráficos relacionados ao comprimento dos comentários em palavras mostram uma distribuição variada de comprimentos através dos diferentes sentimentos. Observando o boxplot, os comentários negativos tendem a ser mais curtos, concentrando-se em torno de um número menor de palavras, enquanto os comentários positivos são tipicamente mais longos, com uma faixa mais ampla de variação no comprimento. Isso pode indicar que usuários tendem a elaborar mais quando expressam satisfação ou elogios. O scatter plot reforça essa observação, mostrando uma maior dispersão nos comentários positivos, indicando variações substanciais no comprimento, enquanto os negativos e neutros apresentam uma aglomeração mais consistente e compacta de dados.

Análise de Comprimento dos Comentários por Sentimento (Caracteres):
Quanto ao comprimento dos comentários em caracteres, a análise reflete padrões similares aos observados com o comprimento em palavras. O bar plot mostra que comentários com sentimentos positivos possuem, em média, mais caracteres, o que é consistente com o padrão de maior extensão observado na análise por palavras. Comentários negativos e neutros apresentam menor extensão média de caracteres. No scatter plot de caracteres, a distribuição por sentimentos também espelha essa tendência, com os comentários positivos mostrando maior dispersão, refletindo uma gama mais ampla de extensões de comentário, do breve ao detalhado.